<a href="https://colab.research.google.com/github/AndersonBlanco/MPS/blob/main/MPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
import PIL
import numpy as np
import random
from random import randint
import cv2
from PIL import Image
import numpy as np
import heapq

In [ ]:
testImg = cv2.imread("/content/MPS/img2.png")
testImg = tf.image.rgb_to_grayscale(testImg)
arr = np.array(testImg)

#gray scale on one cordinate (y,x)
print(arr[75][125])
print(arr.shape)


print(arr[0][0])
plt.figure()
plt.imshow(arr)
plt.show()
#data frame:
data = []
for y in range(arr.shape[0]):
  for x in range(arr.shape[1]):
    data.append([y,x,arr[y][x][0]])

dF = pd.DataFrame(data, columns = ["y", "x", "grayValue"])

print("Data Frame", dF)

#graph data:
plt.figure()
fig, ax = plt.subplots()
plt.scatter(dF["y"], dF["grayValue"], color="blue") #y is our changing value, and "grayValue" is our yielded output per every "y"
plt.legend()
#ax.invert_xaxis()
#ax.invert_yaxis()
plt.show()



In [ ]:


def rgb_to_moon_magnitude(rgb):
    brightness = sum(rgb) / 3  # average brightness for R, G, B
    min_elevation = -9500
    max_elevation = 10500
    magnitude = min_elevation + (brightness / 255) * (max_elevation - min_elevation)
    return magnitude

def image_to_moon_magnitudes(image_path):
    img = Image.open(image_path).convert("RGB")
    img_array = np.array(img)
    magnitudes = np.zeros((img_array.shape[0], img_array.shape[1]))

    for i in range(img_array.shape[0]):
        for j in range(img_array.shape[1]):
            rgb = img_array[i, j]
            magnitudes[i, j] = rgb_to_moon_magnitude(rgb)

    return magnitudes

def heuristic(a, b):
    # Heuristic function for A* (Euclidean distance)
    return np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

def find_optimal_route(elevation_grid, start, end):
    rows, cols = elevation_grid.shape
    open_set = []
    heapq.heappush(open_set, (0, start))

    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, end)}

    while open_set:
        _, current = heapq.heappop(open_set)

        if current == end:
            # Reconstruct path from end to start
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            return path

        neighbors = [
            (current[0] + dx, current[1] + dy)
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]
            if 0 <= current[0] + dx < rows and 0 <= current[1] + dy < cols
        ]

        for neighbor in neighbors:
            elevation_diff = abs(elevation_grid[neighbor] - elevation_grid[current])
            tentative_g_score = g_score[current] + heuristic(current, neighbor) + elevation_diff

            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic(neighbor, end)
                heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return None


def get_optimal_path(image_path, start, end):
    elevation_grid = image_to_moon_magnitudes(image_path)
    path = find_optimal_route(elevation_grid, start, end)
    arr = []
    if path:
        print("Optimal path found:")
        for coord in path:
            x, y = coord
            z = elevation_grid[x, y]
            #print(f"({x}, {y}, {z:.2f})")
            arr.append([x,y,z])
    else:
        print("No path found between the specified points.")
    return arr


image_path = "./MPS/img2.png"
start = (10, 10)  #  start coordinates
end = (75, 150)    #  end coordinates
pathData = get_optimal_path(image_path, start, end)

pathDF = pd.DataFrame(pathData)

plt.figure()
fig, ax = plt.subplots()
ax.invert_yaxis()
plt.imshow(cv2.imread(image_path), cmap="viridis")
plt.plot(pathDF[1], pathDF[0], color = "red")
plt.show()

image_path = "./MPS/img1.png"
start = (50, 50)  # example start coordinates
end = (500, 300)    # example end coordinates
pathData = get_optimal_path(image_path, start, end)
#print(pathData)


pathDF = pd.DataFrame(pathData)

plt.figure()
fig, ax = plt.subplots()
ax.invert_yaxis()
plt.imshow(cv2.imread(image_path), cmap="viridis")
plt.plot(pathDF[1], pathDF[0], color = "red")
plt.show()


